In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from xbbg import blp
import sys
sys.path.append(r'\\10.5.20.31\交易部\Blair\backTestTool')
sys.path.append(r'\\10.5.20.31\交易部\FX trading Strategy souce code\Wilson\程式碼')
import my_function as my
import backtestModuleVer2
from backtestModuleSteveVer import (DrawDown, calculate_detailed_mdd, generate_recent_returns_df, 
                                    print_stats, visualize_portfolio, format_position, 
                                    calculate_portfolio_value,process_portfolio_data)

import sys
from contextlib import contextmanager
import io
import os

# Add this at the beginning of your code, after the imports
strgyName = '1104'
traderName = 'Blair'

devCcy =  ['EUR', 'JPY', 'GBP', 'CHF', 'AUD', 'CAD', 'NZD', 'SEK']
revL = ['NZD', 'EUR', 'AUD', 'GBP']
basket = ['EUR', 'JPY', 'GBP', 'CHF', 'AUD', 'CAD', 'NZD', 'SEK', 'INR', 'IDR', 'ZAR', 'CNH', 'SGD']
#########################################
from datetime import datetime
today = datetime.now()+pd.Timedelta(days=0)
today_str = today.strftime('%Y-%m-%d')
startDate = today-pd.Timedelta(days=600)

# startDate = startDate.strftime('%Y-%m-%d')
startDate = '2012-01-01'
endDate = today_str

dr = pd.date_range(
    start=pd.to_datetime(startDate),
    end=pd.to_datetime(endDate),
    freq=BDay()
)  

def rolling_percentile(data, window=150, min_periods=None):
    """
    calculate rolling percnetile over specific window
    """
    if min_periods is None:
        min_periods = window

    def percentile_rank(window):
        current = window.iloc[-1]
        # print(window, current)
        # print((window <= current).mean())
        return pd.Series([(window <= current).mean() * 100])

    return data.rolling(window=window, min_periods=min_periods).apply(percentile_rank)

def updateW(col):
    count = 0
    pos = 0


    for x in col:
        raw = mri.iloc[count]
        ma = mriMA.iloc[count]

        if (raw > ma):
            if pos == 0:
                # print(f'目前 MRI 為: {raw}，開倉做空風險貨幣')
                pos = -1
            elif pos == 1:
                #print(f'目前 MRI 為: {raw}，關閉做多的風險貨幣')
                pos = 0
        elif (raw < ma) & (pos != 0):
            # if pos == 0:
            #     print(f'目前 MRI 為: {raw}，承平時刻，做多風險貨幣')
            #     pos = 1
            if pos == -1:
                #print(f'目前 MRI 為: {raw}，關閉做空的風險貨幣')
                pos = 0  
        elif (raw < ma) & (pos == 0):
            pass
        col[count] = pos
        count += 1

    return col


if __name__ == '__main__':
    # cur version with rolling percentile: 150, moving average: 20
    df = pd.DataFrame(index=dr)

    # df['US2Y'] = blp.bdh(tickers='USGG2YR Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill()
    # df['US10Y'] = blp.bdh(tickers='USGG10YR Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill()
    # df['us2y10y'] = df['US2Y'] - df['US10Y']

    # df['VIX'] = blp.bdh(tickers='VIX Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill()

    # df['emRet'] = blp.bdh(tickers='M1EF Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill() * -1
    # df['rr'] = blp.bdh(tickers='USDJPY25R1M BGN Curncy', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill() * -1
    # df['fxVol'] = blp.bdh(tickers='CVIX Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate).shift(1).reindex(dr).ffill()


    df['US2Y'] = blp.bdh(tickers='USGG2YR Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate)
    df['US10Y'] = blp.bdh(tickers='USGG10YR Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate)
    df['us2y10y'] = df['US2Y'] - df['US10Y']

    df['VIX'] = blp.bdh(tickers='VIX Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate)

    df['emRet'] = blp.bdh(tickers='M1EF Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate) * -1
    df['rr'] = blp.bdh(tickers='USDJPY25R1M BGN Curncy', flds = ["PX_LAST"], start_date=startDate, end_date=endDate) * -1
    df['fxVol'] = blp.bdh(tickers='CVIX Index', flds = ["PX_LAST"], start_date=startDate, end_date=endDate)


    df = df.iloc[1:, ]
    df.drop(columns=['US2Y', 'US10Y'], inplace=True)
    df.fillna(method='ffill', inplace=True)
    df = df.shift(1)

    df.to_csv(f'{today_str}_MRI_Component.csv')

    roll_Pctl = pd.DataFrame(index=dr, columns=df.columns)
    for asset in roll_Pctl.columns:
        roll_Pctl[asset] = rolling_percentile(df[asset])

    roll_Pctl['simple_avg'] = roll_Pctl.mean(axis=1)

    mri = roll_Pctl['simple_avg'].copy()
    mriMA = mri.rolling(20).mean()

    signalDf = pd.DataFrame(index=dr, columns=basket)
    signalDf = signalDf.apply(updateW, axis=0)
    for ccy in signalDf:
        if ccy not in revL:
            signalDf[ccy] = -1*signalDf[ccy] # 將訊號轉為市場慣例 (USDJPY +1 為 long USDJPY, EURUSD +1 為 long EURUSD)
    signalDf = signalDf * (1/len(basket)) 
    weight_size = 4
    signalDf = signalDf * weight_size
    signalDf.to_csv(f'{strgyName}_Macro_Risk_Signal.csv')



    # Wilson 測試區~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    position_final = signalDf.copy()
    print(f'以下為 Macro Risk ({strgyName}) 策略: ')
    print("近10日部位狀況: ")
    print(position_final.iloc[-10:, :])

    def format_trade_list(condition_summary):
        trade_list = (condition_summary.iloc[-1,] - condition_summary.iloc[-2,])
        # trade_list = (condition_summary.iloc[-1,]) *-1
        formatted_positions = []
        for i in condition_summary.columns:        
            position_value = trade_list[i]
            if position_value == 0:
                continue
            
            
            position_direction = 'Long' if position_value > 0 else 'Short'

            exchange_rate = None
            # If the currency is one of the specified ones, ask for the exchange rate
            if i in ['EUR', 'GBP', 'AUD', 'NZD']:
                exchange_rate = blp.bdh(tickers=f'{i} BGN Curncy', flds = ["PX_LAST"], start_date=datetime.now().strftime('%Y-%m-%d'), end_date=datetime.now().strftime('%Y-%m-%d')).iloc[0, 0]
                position_value /= exchange_rate

            # Format the currency pair based on the given currency
            currency_pair = f"{i}USD" if i in ['EUR', 'GBP', 'AUD', 'NZD'] else f"USD{i}"

            # Prepare the formatted output string
            mio_suffix = f"mio in {i} ({exchange_rate})" if exchange_rate else "mio"
            position_output = f"{position_direction} {currency_pair} {abs(position_value):.2f}{mio_suffix}"
            formatted_positions.append(position_output)
        
        return formatted_positions


    #########################################################################################################################################
    # 輸出的部分
    print("\n")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Wilson 指令輸出區")
    print("還在測試中，請仔細檢查")
    # 使用函數
    formatted_positions = format_trade_list(position_final)
    for position in formatted_positions:
        print(position if position else 'None')

    #########################################################################################################################################
    # 處理信號

    weight_size = 4
    signalDf = pd.read_csv(f'{strgyName}_Macro_Risk_Signal.csv')
    # signalDf['Date'] = signalDf.index
    # last_date = signalDf["Date"].iloc[-1]
    bt = backtestModuleVer2.backTest(MIO=weight_size, signalDf=signalDf, startDate='2012-01-01', endDate=endDate)
    bt.backtestFun()
    bt.strgyStats()

    teStart = '2012-01-01'
    bt.portValChange[bt.portValChange.index >= teStart].to_csv(f'dailyRet_{strgyName}.csv')

    tmpStart = '2025-01-01'
    bt.portValChange[bt.portValChange.index >= tmpStart].to_csv(f'2025_dailyRet_{strgyName}.csv')
    
    # 先處理存儲資料的部分
    daily_total = round((bt.portChange).to_frame(name="Portfolio_Return_Profit"),5)
    daily_total["Portfolio_Return_Profit"] = daily_total["Portfolio_Return_Profit"]*weight_size*1_000_000
    daily_total = daily_total.loc[:today]
    # my.update_excel_save_excel(1104, daily_total, signalDf)
    my.update_excel_save_excel(1104, daily_total)

    daily_total = round((bt.portChange).to_frame(),5)
    daily_total.columns = ["GDP_MoM"]
    daily_total["GDP_MoM"] = daily_total["GDP_MoM"]*weight_size*1_000_000
    daily_total_2024 = daily_total.loc["2024-01-02":today]

    # file_path = r'\\10.5.20.31\交易部\1.周會資料\Tool\FOR0045彙整_Backup.xlsm'
    # mapping_portfolio = {"BKFXOP07DB06TA": "IMM"}
    # df_07 = process_portfolio_data(file_path, mapping_portfolio)

    # 打印统计信息
    print_stats(daily_total_2024['GDP_MoM'], n_days=10)

    # print("\n")
    # visualize_portfolio(daily_total_2024, df_07, 'IMM', 'IMM', start_date, "IMM")
    # print("\n")

    import sys
    sys.path.append(r'\\10.5.20.31\交易部\Blair\strgyOutputPics')